In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#for lemmatization of words we will use nltk library
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

import re                           #regular expression

In [2]:
df = pd.read_csv('data/BA_reviews.csv', index_col=0)
df.head()

,Reviews,Country,Recommended,Rating
0,✅ Trip Verified | This flight was one of the ...,United Kingdom,no,2
1,Not Verified | It seems that there is a race t...,United States,no,3
2,Not Verified | As a Spanish born individual l...,United Kingdom,no,2
3,✅ Trip Verified | A rather empty and quiet fl...,United Kingdom,yes,9
4,✅ Trip Verified | Easy check in and staff mem...,United Kingdom,yes,9


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3461 entries, 0 to 3460
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Reviews      3461 non-null   object
 1   Country      3459 non-null   object
 2   Recommended  3461 non-null   object
 3   Rating       3461 non-null   object
dtypes: object(4)
memory usage: 135.2+ KB


### Checking for Null values

In [4]:
df.isnull().sum()

Reviews        0
Country        2
Recommended    0
Rating         0
dtype: int64

In [5]:
df[df["Country"].isnull()]

,Reviews,Country,Recommended,Rating
2817,I travelled from London to Jo'burg and back on...,NaN,no,2
3117,St Lucia to London round trip. Full flight bot...,NaN,yes,6


### Cleaning `Reviews` column
Removing unnecessary text : For example, "✅ Trip Verified" 

✅ Trip Verified | , Not Verified | , ✅ Verified Review | , ❎ Unverified | 

In [6]:
to_remove_texts = ["✅ Trip Verified | ", "Not Verified | ", "✅ Verified Review | ", "❎ Unverified | "]

reviews = df["Reviews"]

In [7]:
for i in to_remove_texts:
    reviews = reviews.str.replace(i, "", regex=False) 


In [8]:
reviews.head()

0     This flight was one of the worst I have ever ...
1    It seems that there is a race to the bottom am...
2     As a Spanish born individual living in Englan...
3     A rather empty and quiet flight to Tel Aviv, ...
4     Easy check in and staff member was polite and...
Name: Reviews, dtype: object

In [9]:
df["Reviews"] = reviews

In [10]:
df.head()

,Reviews,Country,Recommended,Rating
0,This flight was one of the worst I have ever ...,United Kingdom,no,2
1,It seems that there is a race to the bottom am...,United States,no,3
2,As a Spanish born individual living in Englan...,United Kingdom,no,2
3,"A rather empty and quiet flight to Tel Aviv, ...",United Kingdom,yes,9
4,Easy check in and staff member was polite and...,United Kingdom,yes,9


### Counts for each Rating (1-10)

In [11]:
df['Rating'].value_counts()

1       752
2       392
3       385
8       356
10      315
7       305
9       299
4       237
5       228
6       186
None      6
Name: Rating, dtype: int64

In [12]:
df.drop(df[df['Rating'] == 'None'].index, inplace=True)

### Making Corpus from Reviews

In [15]:

lemma = WordNetLemmatizer()
reviews_data = df['Reviews']

#create an empty list to collect cleaned data corpus
corpus =[]

#loop through each review, remove punctuations, small case it, join it and add it to corpus
for review in reviews_data:
    review = re.sub('[^a-zA-Z]',' ', review)
    review = review.lower()
    review = review.split()
    review = [lemma.lemmatize(word) for word in review if word not in set(stopwords.words("english"))]
    review = " ".join(review)
    corpus.append(review)

In [16]:
# add the corpus to the original dataframe

df['Corpus'] = corpus

In [17]:
df.head()

,Reviews,Country,Recommended,Rating,Corpus
0,This flight was one of the worst I have ever ...,United Kingdom,no,2,flight one worst ever life wanted pamper bough...
1,It seems that there is a race to the bottom am...,United States,no,3,seems race bottom amongst airline cannot even ...
2,As a Spanish born individual living in Englan...,United Kingdom,no,2,spanish born individual living england using b...
3,"A rather empty and quiet flight to Tel Aviv, ...",United Kingdom,yes,9,rather empty quiet flight tel aviv friendly ca...
4,Easy check in and staff member was polite and...,United Kingdom,yes,9,easy check staff member polite helpful made sp...


In [18]:
# export the cleaned data

df.to_csv("data/cleaned_BA_reviews.csv")